In [1]:
import pandas as pd
import geopandas as gpd
from tabula import read_pdf

/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
df_informacoes = read_pdf('data/censo_2010/BASE DE INFORMAåES POR SETOR CENSITµRIO Censo 2010 - Universo.pdf', stream=True, pages=46, lattice=True)

Got stderr: mai 31, 2023 9:58:33 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
ADVERTÊNCIA: Using fallback font 'LiberationSans' for 'TimesNewRomanPSMT'



In [4]:
df_informacoes[0]

,Unnamed: 0,Código da variável,Unnamed: 1,Unnamed: 2,Descrição do setor,Unnamed: 3
0,V003,Média do número de moradores em domicílios\rpa...,NaN,NaN,NaN,NaN
1,V004,Variânciadonúmerodemoradoresem\rdomicílios par...,NaN,NaN,NaN,NaN
2,V005,Valor do rendimento nominal médio mensal das\r...,NaN,NaN,NaN,NaN
3,V006,Variância do rendimento nominal mensal das\rpe...,NaN,NaN,NaN,NaN
4,V007,Valor do rendimento nominal médio mensal das\r...,NaN,NaN,NaN,NaN
5,V008,Variância do rendimento nominal mensal das\rpe...,NaN,NaN,NaN,NaN
6,V009,Valor do rendimento nominal médio mensal das\r...,NaN,NaN,NaN,NaN
7,V010,Variância do rendimento nominal mensal das\rpe...,NaN,NaN,NaN,NaN
8,V011,Valor do rendimento nominal médio mensal das\r...,NaN,NaN,NaN,NaN
9,V012,Variância do rendimento nominal mensal das\rpe...,NaN,NaN,NaN,NaN


In [2]:
## Malha de setores do BR
# https://www.ibge.gov.br/geociencias/downloads-geociencias.html?caminho=organizacao_do_territorio/malhas_territoriais/malhas_de_setores_censitarios__divisoes_intramunicipais/2021/Malha_de_setores_(shp)_Brasil

In [3]:
## Download das tabelas e documentacao
# https://www.ibge.gov.br/estatisticas/sociais/populacao/9662-censo-demografico-2010.html?edicao=10410

# Selecao das variáveis

## Habitação einfraestrutura

* Porcentagem de domicílios do tipo cômodo
* Porcentagem de domicílios sem coleta de lixo
  * Domicilio01_UF.csv, V035
* Porcentagem de domicílios sem ligação à rede de abastecimento de água
  * Domicilio01_UF.csv, V106
* Porcentagem de domicílios sem banheiros ou sanitários
  * Domicilio01_UF.csv, V123
* Porcentagem de domicílios sem ligação à rede de esgoto ou fossa séptica
  * V017 adaptado
* Porcentagem de domicílios – outra forma de posse da moradia
  * N/A
* Número de banheiros por habitante
  * Adaptado V023

## Renda e escolaridade do responsável pelo domicílio

* Porcentagem de responsáveis por domicílio não alfabetizados
  * Responsavel02_UF.csv, adaptado V093
* Porcentagem de responsáveis por domicílio com menos de 30 anos não alfabetizados
* Porcentagem de responsáveis por domicílio com renda de até 3 salários mínimos
  * ResponsavelRenda_UF
* Porcentagem de responsáveis por domicílio com menos de 8 anos de estudo
* Anos médios de estudo do responsável pelo domicílio
* Renda média do responsável pelo domicílio

## Aspectos demográficos

* Número de domicílios particulares permanentes no setor censitário
* Número de domicílios improvisados no setor censitário
* Número de pessoas residentes no setor censitário
* Porcentagem de responsáveis por domicílios com menos de 30 anos
* Número médio de pessoas por domicílio



In [4]:
gdf_br = gpd.read_file('data/censo_2010/BR_Setores_2021.zip')

In [5]:
gdf_br.shape

(452248, 13)

In [6]:
gdf_br.NM_SIT.value_counts()

Área Urbana de Alta Densidade de Edificações     306095
Área Rural (exclusive aglomerados)                83569
Área Urbana de Baixa Densidade de Edificações     31869
Povoado                                           16832
Núcleo Urbano                                      7682
Lugarejo                                           5010
Massa de Água                                       894
Núcleo Rural                                        295
Name: NM_SIT, dtype: int64

In [7]:
gdf_br.columns

Index(['CD_SETOR', 'CD_SIT', 'NM_SIT', 'CD_UF', 'NM_UF', 'SIGLA_UF', 'CD_MUN',
       'NM_MUN', 'CD_DIST', 'NM_DIST', 'CD_SUBDIST', 'NM_SUBDIST', 'geometry'],
      dtype='object')

In [3]:
UFS = [
    'AC',
    'AL',
    'AM',
    'AP',
    'BA',
    'CE',
    'DF',
    'ES',
    'GO',
    'MA',
    'MG',
    'MS',
    'MT',
    'PA',
    'PB',
    'PE',
    'PI',
    'PR',
    'RJ',
    'RN',
    'RO',
    'RR',
    'RS',
    'SC',
    'SE',
    'SP1',
    'SP2',
    'TO'
]

In [25]:
## Hash para montar a base de dados
# {'file1': ['var1', 'var2', ..., 'var n'], ...,'file n': ['var1', 'var2', ..., 'var n']}

model_vars = {
    'Basico':{
        # 'Cod_Grandes Regiões': 'Código das GRandes REgiões',
        # 'Nome_Grande_Regiao': 'Nome GRande Região',
        'Cod_UF': 'uf',
        'Cod_RM': 'rm',
        'Nome_da_RM': 'nome_rm',
        'Cod_municipio': 'cod_municipio',
        'Nome_do_municipio': 'nome_municipio',
        'Situacao_setor': 'situacao_setor',
        'Tipo_setor': 'tipo_do_setor',
        'V001': 'numero_de_domicilios',
        'V002': 'numero_moradores'
    },
    'Domicilio01': {
        'V003': 'domicilios_tipo_casa',
        'V002': 'domicilios_particulares_permanentes',
        'V035': 'domicilios_com_lixo_coletado',
        'V106': 'domicilios_tipo_casa_com_abastecimento_de_agua',
        'V023': 'domicilios_particulares_permanentes_sem_banheiro_ou_sanitario',
        'V017': 'domicilios_particulares_permanentes_com_banheiro_ligado_a_esgoto',
        'V018': 'domicilios_particulares_permanentes_com_banheiro_ligado_a_fossa_septica',
        'V025': 'domicilios_particulares_permanentes_com_1_banheiro',
        'V026': 'domicilios_particulares_permanentes_com_2_banheiros',
        'V027': 'domicilios_particulares_permanentes_com_3_banheiros',
        'V028': 'domicilios_particulares_permanentes_com_4_banheiros',
        'V029': 'domicilios_particulares_permanentes_com_5_banheiros',
        'V030': 'domicilios_particulares_permanentes_com_6_banheiros',
        'V031': 'domicilios_particulares_permanentes_com_7_banheiros',
        'V032': 'domicilios_particulares_permanentes_com_8_banheiros',
        'V033': 'domicilios_particulares_permanentes_com_mais_de_9_banheiros',
        'V010': 'domicilios_particulares_permanentes_cedidos_de_outra_forma',
        'V011': 'domicilios_particulares_permanentes_em_outra_condicao_de_ocupacao',
    },
    'Responsavel02': {
        'V001': 'pessoas_responsaveis',
        'V002': 'pessoas_responsaveis_com_10_anos',
        'V003': 'pessoas_responsaveis_com_11_anos',
        'V004': 'pessoas_responsaveis_com_12_anos',
        'V005': 'pessoas_responsaveis_com_13_anos',
        'V006': 'pessoas_responsaveis_com_14_anos',
        'V007': 'pessoas_responsaveis_com_15_anos',
        'V008': 'pessoas_responsaveis_com_16_anos',
        'V009': 'pessoas_responsaveis_com_17_anos',
        'V010': 'pessoas_responsaveis_com_18_anos',
        'V011': 'pessoas_responsaveis_com_19_anos',
        'V012': 'pessoas_responsaveis_com_20_anos',
        'V013': 'pessoas_responsaveis_com_21_anos',
        'V014': 'pessoas_responsaveis_com_22_anos',
        'V015': 'pessoas_responsaveis_com_23_anos',
        'V016': 'pessoas_responsaveis_com_24_anos',
        'V017': 'pessoas_responsaveis_com_25_anos',
        'V018': 'pessoas_responsaveis_com_26_anos',
        'V019': 'pessoas_responsaveis_com_27_anos',
        'V020': 'pessoas_responsaveis_com_28_anos',
        'V021': 'pessoas_responsaveis_com_29_anos',
        'V093': 'pessoas_responsaveis_alfabetizadas',
        'V094': 'pessoas_responsaveis_alfabetizadas_de_10_a_14_anos',
        'V095': 'pessoas_responsaveis_alfabetizadas_de_15_a_19_anos',
        'V096': 'pessoas_responsaveis_alfabetizadas_de_20_a_24_anos',
        'V097': 'pessoas_responsaveis_alfabetizadas_de_25_a_29_anos'
    },
    'DomicilioRenda':{
        'V001': 'total_domicilios_improvisados',
        'V002': 'total_rendimento_dos_domicilios',
    },
    'ResponsavelRenda':{
        'V001': 'responsaveis_com_renda_ate_meio_salario_minimo',
        'V002': 'responsaveis_com_renda_de_meio_a_1_salario_minimo',
        'V003': 'responsaveis_com_renda_de_1_a_2_salarios_minimos',
        'V004': 'responsaveis_com_renda_de_2_a_3_salarios_minimos'
    }
}


In [26]:
dfs = []
for uf in UFS:
    print(f'Processando {uf}')
    dfs_uf = []
    for file in model_vars:
        file_path = f'data/censo_2010/csvs/{file}_{uf}.csv'
        fields = list(model_vars[file].keys())
        fields.append('Cod_setor')
        try:
            df = pd.read_csv(file_path, delimiter=';', usecols=fields, dtype="string", encoding='ISO-8859-1').set_index('Cod_setor')
        except:
            try:
                df = pd.read_csv(file_path, delimiter=',', usecols=fields, dtype="string", encoding='ISO-8859-1').set_index('Cod_setor')
            except:
                df = pd.read_csv(file_path, delimiter=';', usecols=fields, dtype="string", encoding='UTF-8').set_index('Cod_setor')
        df.rename(columns=model_vars[file], inplace=True)
        dfs_uf.append(df)
        # break
    df = pd.concat(dfs_uf, axis=1)
    dfs.append(df)

Processando AC
Processando AL
Processando AM
Processando AP
Processando BA
Processando CE
Processando DF
Processando ES
Processando GO
Processando MA
Processando MG
Processando MS
Processando MT
Processando PA
Processando PB
Processando PE
Processando PI
Processando PR
Processando RJ
Processando RN
Processando RO
Processando RR
Processando RS
Processando SC
Processando SE
Processando SP1
Processando SP2
Processando TO


In [27]:
pd.concat(dfs)

,uf,rm,nome_rm,cod_municipio,nome_municipio,situacao_setor,tipo_do_setor,numero_de_domicilios,numero_moradores,domicilios_particulares_permanentes,...,pessoas_responsaveis_alfabetizadas_de_10_a_14_anos,pessoas_responsaveis_alfabetizadas_de_15_a_19_anos,pessoas_responsaveis_alfabetizadas_de_20_a_24_anos,pessoas_responsaveis_alfabetizadas_de_25_a_29_anos,total_domicilios_improvisados,total_rendimento_dos_domicilios,responsaveis_com_renda_ate_meio_salario_minimo,responsaveis_com_renda_de_meio_a_1_salario_minimo,responsaveis_com_renda_de_1_a_2_salarios_minimos,responsaveis_com_renda_de_2_a_3_salarios_minimos
Cod_setor,,,,,,,,,,,,,,,,,,,,,
120001305000001,12,00,Municípios não pertencentes a estrutura de RM,1200013,ACRELÂNDIA,1,0,282,957,282,...,0,2,19,23,0,601805,17,89,63,25
120001305000002,12,00,Municípios não pertencentes a estrutura de RM,1200013,ACRELÂNDIA,1,0,342,1203,342,...,0,9,24,50,2,385433,30,139,102,19
120001305000003,12,00,Municípios não pertencentes a estrutura de RM,1200013,ACRELÂNDIA,1,0,482,1700,482,...,2,18,47,72,1,532208,42,193,98,28
120001305000004,12,00,Municípios não pertencentes a estrutura de RM,1200013,ACRELÂNDIA,5,0,53,182,53,...,0,1,3,7,0,58853,3,14,21,8
120001305000005,12,00,Municípios não pertencentes a estrutura de RM,1200013,ACRELÂNDIA,8,0,96,305,96,...,0,0,3,13,0,112979,6,47,25,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170210905000098,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,359,...,0,4,33,63,0,484164,13,100,162,35
170210905000099,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,354,...,0,6,23,56,1,341945,11,165,85,16
170210905000100,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,338,...,0,4,21,52,0,441825,10,178,77,25
